# HEADER HERE
Text
- bullet points

In [6]:
#Script to get folding energies and transcript sequences using genes from DESeq2 output
from Bio import SeqIO
from Bio.Seq import Seq
import RNA

#Use tx2gene file used for DESeq2, since DESeq2 output is in common name rather than SGD IDs
TxToGeneFile = "tx2gene.txt"

#Use fasta file generated using SGD R64-1-1 CDSes and Pelechano UTR annotation
#Uses longest UTR if available or default 5' and 3'UTRs of 120 nt and 200 nt, respectively, if none found
TranscriptomeFile = "R64-1-1_transcriptome_with_utrs.fasta"

#return dictionary of transcript sequences mapped to common names
transcripts = {}

#Build dictionary of common names mapped to SGD IDs
txToGene = {}
with open(txToGeneFile, "r") as inFile:
        for line in conversionFile:
            #Skip the first line
            if ("GENEID" not in line):
                data = line.strip().split("\t")
                transcriptId = data[0]
                geneId = data[1]
                txToGene[geneId] = transcriptId
            
    #Build dictionary of transcript sequences mapped to SGD ID
    transcripts = {}
    transcriptFasta = SeqIO.parse(open(transcriptFile), 'fasta')
    for fasta in transcriptFasta:
        name = geneToTx()
        sequence = str(fasta.seq)
        if (asRNA):
            sequence = sequence.replace("T", "U")
        transcripts[fasta.name] = sequence
        
    return transcripts

def getStremeFasta(geneList, outputFile, asRNA = True, txToGeneFile = defaultTxToGene, transcriptFile = defaultTranscriptome):
    transcriptList = genesToTranscripts(asRNA, txToGeneFile, transcriptFile)
    
    #Parse gene list to write out transcript sequences
    with open(geneList, "r") as inFile, open(outputFile, "w") as outFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = transcriptList[gene]
            #get transcript sequence
            transcript = transcripts[sgdId]
            #write to fasta
            outFile.write(">" + sgdId + "\n")
            outFile.write(transcript + "\n")
    
        #Get gene list to write out transcript sequences
    with open(geneList, "r") as inFile, open(outputFile, "w") as outFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = geneToTx[gene]
            #get transcript sequence
            transcript = transcripts[sgdId]
            #write to fasta
            outFile.write(">" + sgdId + "\n")
            outFile.write(transcript + "\n")
    
    
    
    
    
    
    
    #Build dictionary of transcript sequences for the given gene list
    outputTranscripts = {}
    with open(geneList, "r") as inFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = geneToTx[gene]
            transcriptSeq = transcripts[sgdId]
            if (asRNA):
                transcriptSeq = transcriptSeq.replace("T", "U")
            outputTranscripts[sgdId] = transcriptSeq
            
    return transcripts
        
def getStremeFasta(geneList, asRNA = True, txToGeneFile = defaultTxToGene, transcriptFile = defaultTranscriptome):
        #Get gene list to write out transcript sequences
    with open(geneList, "r") as inFile, open(outputFile, "w") as outFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = geneToTx[gene]
            #get transcript sequence
            transcript = transcripts[sgdId]
            #write to fasta
            outFile.write(">" + sgdId + "\n")
            outFile.write(transcript + "\n")
            
    
def getFoldingEnergies(geneList, asRNA = True, txToGeneFile = defaultTxToGene, transcriptFile = defaultTranscriptome):
        #Get gene list to write out transcript sequences
    with open(geneList, "r") as inFile, open(outputFile, "w") as outFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = geneToTx[gene]
            #get transcript sequence
            transcript = transcripts[sgdId]
            #write to fasta
            outFile.write(">" + sgdId + "\n")
            outFile.write(transcript + "\n")
            
    
    with open("tx2gene.txt", "r") as tx2gene_file, open("TE Multivariate List.txt", "r") as in_file:
    for line in tx2gene_file:
        if ("GENEID" not in line):
            data = line.strip().split("\t")
            tx2gene[data[1]] = data[0]
            
    for line in in_file:
        genes.append(tx2gene[line.strip()])

transcriptome = {}
with open("TE_dG.txt", "w") as out_file:        
    transcript_file = SeqIO.parse(open("R64-1-1_transcriptome_with_utrs.fasta"), 'fasta')
    for fasta in transcript_file:
        sequence = str(fasta.seq)
        sequence = sequence.replace("T", "U")
        transcriptome[fasta.name] = sequence
        
    for gene in genes:
        (ss, mfe) = RNA.fold(transcriptome[gene])
        out_file.write(gene + "\t" + str(mfe) + "\n")
    


def getStremeFasta(txToGeneFile, geneList, transcriptFile, outputFile):
    
    #Build dictionary of SGD IDs mapped to common names
    geneToTx = {}
    with open(txToGeneFile, "r") as conversionFile:
        for line in conversionFile:
            #Skip the first line
            if ("GENEID" not in line):
                data = line.strip().split("\t")
                geneToTx[data[1]] = data[0]
            
    #Build dictionary of transcript sequences mapped to SGD ID
    transcripts = {}
    transcriptFasta = SeqIO.parse(open(transcriptFile), 'fasta')
    for fasta in transcriptFasta:
        sequence = str(fasta.seq)
        #replace all T's with U's to convert DNA to RNA
        sequence = sequence.replace("T", "U")
        transcripts[fasta.name] = sequence
        
    #Get gene list to write out transcript sequences
    with open(geneList, "r") as inFile, open(outputFile, "w") as outFile:
        for line in inFile:
            gene = line.strip()
            #convert common name to SGD ID
            sgdId = geneToTx[gene]
            #get transcript sequence
            transcript = transcripts[sgdId]
            #write to fasta
            outFile.write(">" + sgdId + "\n")
            outFile.write(transcript + "\n")
            

#Get fasta file for TEs
getStremeFasta(defaultTxToGene, "Up in TE.txt", defaultTranscriptome, "Up in TE.fasta")
#Get fasta file for ribosome profiling
getStremeFasta(defaultTxToGene, "Up in RPF.txt", defaultTranscriptome, "Up in RPF.fasta")
#Get fasta file for RNA-seq
getStremeFasta(defaultTxToGene, "Up in RNA.txt", defaultTranscriptome, "Up in RNA.fasta")

In [ ]:
#Script to get folding energies for transcripts using ViennaRNA RNAFold

from Bio import SeqIO
from Bio.Seq import Seq
import RNA

def getFoldingEnergies(txToGeneFile, geneList, transcriptFile, outputFile):
    


tx2gene = {}
genes = []

with open("tx2gene.txt", "r") as tx2gene_file, open("TE Multivariate List.txt", "r") as in_file:
    for line in tx2gene_file:
        if ("GENEID" not in line):
            data = line.strip().split("\t")
            tx2gene[data[1]] = data[0]
            
    for line in in_file:
        genes.append(tx2gene[line.strip()])

transcriptome = {}
with open("TE_dG.txt", "w") as out_file:        
    transcript_file = SeqIO.parse(open("R64-1-1_transcriptome_with_utrs.fasta"), 'fasta')
    for fasta in transcript_file:
        sequence = str(fasta.seq)
        sequence = sequence.replace("T", "U")
        transcriptome[fasta.name] = sequence
        
    for gene in genes:
        (ss, mfe) = RNA.fold(transcriptome[gene])
        out_file.write(gene + "\t" + str(mfe) + "\n")